In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
import itertools
from scipy.interpolate import interp1d
from helper import *

In [4]:
%matplotlib inline
%config IPython.matplotlib.backend = "retina"
from matplotlib import rcParams
rcParams["figure.dpi"] = 300
rcParams["savefig.dpi"] = 300
rcParams["xtick.direction"] = "in"
rcParams["ytick.direction"] = "in"
rcParams["xtick.top"] = True
rcParams["ytick.right"] = True
rcParams["font.size"] = 14

In [5]:
var_name = "mesh_delta_coeff"
var_samples = np.arange(0.1, 1.7, 0.2)

M_sample = [1.7]
Z_sample = [0.015]
V_sample = [0]

log_dirs = glob.glob("test/m1.7_z0.015_v0_var*/LOGS")
run_times = []
var_ind = []
for i, logs_dir in enumerate(log_dirs):
    var_ind.append(int(logs_dir.split("/LOGS")[0].split("var")[-1]))
    logfn = "".join(logs_dir.split("LOGS")) + "/run.log"
    with open(logfn, "r") as f:
        lines = f.readlines()
        for line in lines:
            if "Total" in line:
                run_time = float(line.split(" ")[-2])
                run_times.append(run_time)
var_samples = [round(var_samples[i-1], 2) for i in var_ind]

combinations = itertools.product(M_sample, Z_sample, V_sample, var_samples)

print(run_times)
print(var_samples)
print(var_ind)

[
    3371.7506079673767,
    1670.9988837242126,
    1031.0434262752533,
    817.9263074398041,
    693.2565948963165,
    652.3848984241486,
    509.527375459671,
    461.2132480144501
]

[0.1, 0.3, 0.5, 0.7, 0.9, 1.1, 1.3, 1.5]

[1, 2, 3, 4, 5, 6, 7, 8]

In [43]:
def get_freqs(ind):
    var = var_samples[ind]
    hist, freqs, profs, n_profs, prof_index = get_data(log_dirs[ind])
    models = [i[0] for i in prof_index if i[2] in n_profs]
    indexes = hist.index[hist.model_number.isin(models)]
    ages = hist.star_age[indexes]/1e6
    return freqs, ages

def interp(x, y, age):
    f = interp1d(x, y, fill_value="extrapolate")
    return f(age)

mode_labels = ["n1ell0m0","n2ell0m0","n3ell0m0","n4ell0m0","n5ell0m0","n6ell0m0","n7ell0m0","n8ell0m0","n9ell0m0","n10ell0m0",
         "n1ell1mm1","n2ell1mm1","n3ell1mm1","n4ell1mm1","n5ell1mm1","n6ell1mm1","n7ell1mm1","n8ell1mm1","n9ell1mm1","n10ell1mm1",
         "n1ell1m0","n2ell1m0","n3ell1m0","n4ell1m0","n5ell1m0","n6ell1m0","n7ell1m0","n8ell1m0","n9ell1m0","n10ell1m0",
         "n1ell1mp1","n2ell1mp1","n3ell1mp1","n4ell1mp1","n5ell1mp1","n6ell1mp1","n7ell1mp1","n8ell1mp1","n9ell1mp1","n10ell1mp1"]
mode_strings = ["100","200","300","400","500","600","700","800","900","1000",
         "11-1","21-1","31-1","41-1","51-1","61-1","71-1","81-1","91-1","101-1",
         "110","210","310","410","510","610","710","810","910","1010",
         "11-1","21-1","31-1","41-1","51-1","61-1","71-1","81-1","91-1","101-1"]

def get_freqs_df_interp(ind):
    freqs, ages = get_freqs(ind)
    dff = pd.DataFrame()
    dff["Dnu"] = np.repeat(np.nan, len(freqs))
    dff["eps"] = np.repeat(np.nan, len(freqs))
    for s in mode_labels:
        dff[s] = np.repeat(np.nan, len(freqs))

    freqs_df = []
    for i, row in dff.iterrows():
        df = row
        ts = freqs[i]
        ts = model_nlm(ts)
        df["age"] = ages.iloc[i]
        df["Dnu"] = model_Dnu(ts)
        df["eps"] = model_epsilon(ts)
        for j,s in enumerate(mode_strings):
            try:
                df[mode_labels[j]] = np.round(ts.query(f"nlm=='{s}'")["Re(freq)"].values[0],5)
            except:
                df[mode_labels[j]] = np.nan
        freqs_df.append(df)

    freqs_df = pd.DataFrame(freqs_df)

    l = 1
    for n in range(1, 11):
        freqs_df[f"n{n}ell{l}dfreq"] = freqs_df[f"n{n}ell{l}m0"] - freqs_df[f"n{n}ell{l}mm1"]
        freqs_df.drop(columns=[f"n{n}ell{l}mp1", f"n{n}ell{l}mm1"], inplace=True)
    columns = ['age', 'Dnu', 'eps', 'n1ell0m0', 'n2ell0m0', 'n3ell0m0', 'n4ell0m0',
       'n5ell0m0', 'n6ell0m0', 'n7ell0m0', 'n8ell0m0', 'n9ell0m0', 'n10ell0m0',
       'n1ell1m0', 'n2ell1m0', 'n3ell1m0', 'n4ell1m0', 'n5ell1m0', 'n6ell1m0',
       'n7ell1m0', 'n8ell1m0', 'n9ell1m0', 'n10ell1m0', 'n1ell1dfreq',
       'n2ell1dfreq', 'n3ell1dfreq', 'n4ell1dfreq', 'n5ell1dfreq',
       'n6ell1dfreq', 'n7ell1dfreq', 'n8ell1dfreq', 'n9ell1dfreq',
       'n10ell1dfreq']
    freqs_df = freqs_df[columns]
    # return freqs_df
    
    ages = np.arange(0, freqs_df.iloc[-1]["age"], 0.1)
    new_df = pd.DataFrame()
    new_df["age"] = ages
    for column in columns[1:]:
        new_df[column] = interp(freqs_df["age"], freqs_df[column], ages)
    return new_df
    

In [44]:
combinations = itertools.product(M_sample, Z_sample, V_sample, var_samples)
data_dict = {}

k = 0
for m, z, v, var in combinations:
    print(m, z, v, var)
    key = f"M={m}_Z={z}_V={v}_var={var}"
    hist, freqs, profs, n_profs, prof_index = get_data(log_dirs[k])
    data_dict[key] = pd.DataFrame()
    data_dict[key]['age'] = hist['star_age']/1e6
    data_dict[key]['log_cntr_Rho'] = hist['log_cntr_Rho'].values
    data_dict[key]['log_cntr_T'] = hist['log_cntr_T'].values
    data_dict[key]['log_Teff'] = hist['log_Teff'].values
    data_dict[key] = pd.merge(data_dict[key], get_freqs_df_interp(k), on="age")
    k += 1

1.7 0.015 0 0.1

In [22]:
data_dict.keys()

dict_keys(['M=1.7_Z=0.015_V=0_var=0.1', 'M=1.7_Z=0.015_V=0_var=0.3', 'M=1.7_Z=0.015_V=0_var=0.5', 'M=1.7_Z=0.015_V=0_var=0.7', 'M=1.7_Z=0.015_V=0_var=0.9', 'M=1.7_Z=0.015_V=0_var=1.1', 'M=1.7_Z=0.015_V=0_var=1.3', 'M=1.7_Z=0.015_V=0_var=1.5'])

In [42]:
# data_dict['M=1.7_Z=0.015_V=0_var=0.1'].iloc[0][6:16]
data_dict['M=1.7_Z=0.015_V=0_var=0.1'].iloc[0][17:26].values

10.70737

In [40]:
key_ref = "M=1.7_Z=0.015_V=0_var=0.1"
mean_freqs_diff = []
for key in data_dict.keys():
    for i, row in data_dict[key].iterrows():
        freqs_alt_0 = row[6:16].values
        freqs_alt_1 = row[17:26].values
        freqs_ref_0 = data_dict[key_ref].iloc[i][6:16].values
        freqs_ref_1 = data_dict[key_ref].iloc[i][17:26].values
        diff_0 = (freqs_ref_0 - freqs_alt_0)/freqs_ref_0
        diff_1 = (freqs_ref_1 - freqs_alt_1)/freqs_ref_1
        mean_freqs_diff.append( np.mean(np.append(diff_0, diff_1)) )

IndexError: single positional indexer is out-of-bounds

In [ ]:
fig = plt.figure(figsize=(15, 15))

ax1 = fig.add_subplot(2, 1, 1)
